## Calibration loop

In [ ]:
import os
from du_astro_utils import calibration, photometry, utils
from tqdm import tqdm

In [ ]:
target_type = "variable_star"
if target_type == "asteroid":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_ASTER)
elif target_type == "exoplanet":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_EXPLTS)
elif target_type == "variable_star":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_VARSTARS)
elif target_type == "galaxy_cluster":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_GALCLUST)
elif target_type == "cluster":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_CLUSTERS)
os.listdir(data_dir)

In [ ]:
for ddir in os.listdir(data_dir):
    subdata_dir = os.path.join(data_dir, ddir)
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
        list_fits = sorted(list_fits)
        print(subdata_dir, len(list_fits))

In [ ]:
achoice = os.listdir(data_dir)[2]
subdata_dir = os.path.join(data_dir, achoice)
list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
list_fits = sorted(list_fits)

In [ ]:
rerun = False
test_mode = False  # Only runs on a few images, prints the tables at each step and does not write files.
dict_of_dict = {}

if test_mode:
    list_fits = list_fits[:10]
if rerun:
    for loc, scimage in enumerate(tqdm(list_fits)):
        fits_sci_image = os.path.join(subdata_dir, scimage)
        fits_sci_image = os.path.abspath(fits_sci_image)
        if os.path.isfile(fits_sci_image):
            # Get image directory, failename and extension
            sc_im_dir = os.path.abspath(os.path.dirname(fits_sci_image))
            sc_im_name, sc_im_ext = os.path.splitext(os.path.basename(fits_sci_image))

            bias_dir, darks_dir, flats_dir = utils.get_calib_dirs_photometry(fits_sci_image)

            # Get information from FITS header
            sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y = calibration.get_infos_from_image(fits_sci_image, verbose=False)
            # print(sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y)

            # Run calibration
            dico_calib = calibration.dedark_sci_image(fits_sci_image, override_date_check=True, max_days=7, overwrite=rerun, verbose=test_mode, write_tmp=test_mode, overwrite_calibs=False)
            dict_of_dict.update({sc_im_name: dico_calib["path"]})

**Plate-solve and align images and perform multi-aperture astrometry with AIJ !**
1. Plate-solve all images from the virtual stack
2. Align using WCS
3. Load the aligned images in a new virtual stack
4. Plot aperture profile, save to apertures
5. Multi-aperture photometry with auto-identified reference stars.
6. Save all measurements and apertures in a `fits.fz` file. Save plot as `PNG`. Save measurements table as `fits`.

In [ ]:
from astropy.table import Table

In [ ]:
aperphot = Table.read("Cy-Aqr_Measurements.fits")

In [ ]:
aperphot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d

In [ ]:
aperphot["T_MIN"] = (aperphot["J.D.-2400000"] - aperphot["J.D.-2400000"][0]) * 24 * 60

In [ ]:
signal = gaussian_filter1d(aperphot["rel_flux_T1"], 7)
plt.scatter(aperphot["T_MIN"], aperphot["rel_flux_T1"])
plt.plot(aperphot["T_MIN"], signal, c="r")

In [ ]:
vmax, vmin = np.max(signal), np.min(signal)
amp = 0.5 * (vmax - vmin)
print(f"$\lambda_0 = ${amp:.4g}")
vmoy = np.mean(signal)
print(f"$\mu_0 = ${vmoy:.4g}")
t_min, t_max = aperphot["T_MIN"][np.argmin(signal)], aperphot["T_MIN"][np.argmax(signal)]
T = 2 * np.abs(t_min - t_max)
print(f"$\\tau_0 = ${T:.4g}")


def sin_to_fit(t, A, B, C, dt):
    return vmoy + A + np.sin(2 * np.pi * (t + dt) / (C * T)) * amp * B

In [ ]:
T_sel = np.logical_or(aperphot["T_MIN"] < 30, np.logical_and(aperphot["T_MIN"] > 80, aperphot["T_MIN"] < 120))

In [ ]:
sin_coeffs = curve_fit(sin_to_fit, aperphot["T_MIN"][T_sel], aperphot["rel_flux_T1"][T_sel], sigma=aperphot["rel_flux_err_T1"][T_sel], maxfev=10000)[0]

In [ ]:
plt.scatter(aperphot["T_MIN"][T_sel], aperphot["rel_flux_T1"][T_sel], alpha=0.4, label="AIJ aperture photometry-\n selected points for fit")
plt.scatter(aperphot["T_MIN"][np.logical_not(T_sel)], aperphot["rel_flux_T1"][np.logical_not(T_sel)], alpha=0.4, label="AIJ aperture photometry-\n other points")
plt.plot(aperphot["T_MIN"], signal, c="r", ls=":", label="Smoothed data")
PER_min = sin_coeffs[2] * T
plt.plot(aperphot["T_MIN"], sin_to_fit(aperphot["T_MIN"], *sin_coeffs), c="g", label=f"Fitted sine : T={PER_min:.2f} min")
plt.legend()
plt.xlabel("$\Delta T$ [min]")
plt.ylabel("rel_flux_T1")
plt.grid()

In [ ]:
PER_min = sin_coeffs[2] * T * 2
print(f"Période estimée : {PER_min:.2f} minutes")
hh = PER_min // 60
mm = PER_min % (hh * 60)
print(f"Période estimée : {hh:.0f}h{mm:.0f}m")

In [ ]:
print("A={:.4g}, B={:.4g}, C={:.4g}, dt={:.4g}".format(*sin_coeffs))

#### Cy-Aqr

In [ ]:
PER_ref_min = 0.061038408 * 24 * 60
print(f"Période référence : {PER_ref_min:.2f} minutes")
hhref = PER_ref_min // 60
mmref = PER_ref_min % (hhref * 60)
print(f"Période référence : {hhref:.0f}h{mmref:.0f}m")

In [ ]:
err = (PER_min - PER_ref_min) / PER_ref_min
print(f"Erreur : {err*100:.1f}%")